# 02_그룹분석

그룹분석이란 "학년별 국어평균"과 같이 데이터를 집단별로 나누어 그룹함수를 적용한 분석 방법

ex) 지역별 인구수,  학년별 평균점수, 연령별 평균소득

In [1]:
from pandas import DataFrame
from pandas import read_csv
from matplotlib import pyplot
from pandas import merge
import numpy as np

In [2]:
df = read_csv("grade.csv", encoding="euc-kr")
df

,이름,학년,성별,국어,영어,수학,과학
0,철수,1,남자,98,NaN,88.0,64.0
1,영희,2,여자,88,90.0,62.0,72.0
2,민수,1,남자,92,70.0,NaN,NaN
3,수현,3,여자,63,60.0,31.0,70.0
4,호영,4,남자,120,50.0,NaN,88.0


In [3]:
이름 = list(df['이름'])
dic = {}
for i, v in enumerate(이름):
    dic[i] = v
df.rename(index=dic, inplace=True)
df.drop(['이름'], axis=1, inplace=True)
df

,학년,성별,국어,영어,수학,과학
철수,1,남자,98,NaN,88.0,64.0
영희,2,여자,88,90.0,62.0,72.0
민수,1,남자,92,70.0,NaN,NaN
수현,3,여자,63,60.0,31.0,70.0
호영,4,남자,120,50.0,NaN,88.0


## #01. 집계함수 사용

> 데이터프레임에서 **집계함수**를 사용하면 **열 단위(세로방향)** 으로 수행 된다.

### 사용 가능한 집계함수의 종류

- 그룹별 데이터의 개수 : size, count
- 평균, 중앙값, 최소, 최대 : mean, median, min, max
- 합계, 곱, 표준편차, 분산, 사분위 수 : sum, prod, std, var, quantile
- 그룹 안에서 첫번째, 마지막 데이터 호출 : first, last

### 1) 전체 열에 대한 집계

#### 총점 (sum)

> 데이터 타입이 `문자열`인 컬럼에 대해서는 모든 데이터를 **하나의 문자열로 병합한 값** 이 표시 된다.

In [4]:
# 과목별 총점
df.sum()

학년            11
성별    남자여자남자여자남자
국어           461
영어         270.0
수학         181.0
과학         294.0
dtype: object

#### 평균(mean)

> - 숫자형태가 아니기 때문에 값을 계산 할 수 없는 컬럼은 **자동으로 제외**된다.
> - `numeric_only=True` 파라미터을 사용

In [5]:
df.mean(numeric_only=True)

학년     2.200000
국어    92.200000
영어    67.500000
수학    60.333333
과학    73.500000
dtype: float64

### 2) 특정열에 대한 집계

In [6]:
df['국어'].max()

120

### 3) 행 단위 집계

> 집계함수에 `axis=1` 파라미터 지정
> **집계함수의 axis의 의미**
 - axis=0 : y축, 열 단위,  기본값
 - axis=1 : x축, 행 단위
 - **`drop()` 함수와 반대 개념**으로 동작한다.
 -  numeric_only=True 사용

#### 학생별 총점

In [7]:
df.sum(axis=1, numeric_only=True)

철수    251.0
영희    314.0
민수    163.0
수현    227.0
호영    262.0
dtype: float64

#### 특정 행에 대한 집계

In [8]:
# 영희 평균을 구하자
p1 = df.filter(['국어','영어','수학','과학'])
p1_avg = p1.loc["영희"].mean()
p1_avg

78.0

In [9]:
# 민수가 nan은 포함하지 않는다.
p2 = df.filter(['국어','영어','수학','과학'])
p2_avg = p2.loc["민수"].mean()
p2_avg

81.0

### 4) 행 단위 집계 결고라르 새로운 열로 추가하기 

> **`axis` 파라미터값이 `drop()`함수와 반대로 적용됨**

In [10]:
df2 = df.filter(['국어', '영어','수학','과학'])
df2

,국어,영어,수학,과학
철수,98,NaN,88.0,64.0
영희,88,90.0,62.0,72.0
민수,92,70.0,NaN,NaN
수현,63,60.0,31.0,70.0
호영,120,50.0,NaN,88.0


In [11]:
s1 = df2.sum(axis=1)
s2 = df2.mean(axis=1)

df2["총점"] = s1
df2["평균"] = s2
df2

,국어,영어,수학,과학,총점,평균
철수,98,NaN,88.0,64.0,250.0,83.333333
영희,88,90.0,62.0,72.0,312.0,78.000000
민수,92,70.0,NaN,NaN,162.0,81.000000
수현,63,60.0,31.0,70.0,224.0,56.000000
호영,120,50.0,NaN,88.0,258.0,86.000000


##  #03. 집단별로 나누기

- 동일한 값을 갖는 데이터들 끼리 그룹으로 묶고, 그 이외의 다른 데이터들에게  집계를 수행하는 형태.
- SQL의 group by 절과 같은 기능

### 1) 샘플 데이터 가져오기

In [12]:
인구조사 = read_csv("city_people.csv", encoding="euc-kr")
인구조사

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,2632035,수도권


#### 도시와 인구 추출

In [13]:
도시별인구 = 인구조사.filter(['도시','인구'])
도시별인구

,도시,인구
0,서울,9904312
1,서울,9631482
2,서울,9762546
3,부산,3448737
4,부산,3393191
5,부산,3512547
6,인천,2890451
7,인천,2632035


#### 도시끼리 그룹화 하자 

> 집계함수사용 : sum, mean, median, min, max

In [14]:
도시별그룹 = 도시별인구.groupby('도시').sum()
도시별그룹

,인구
도시,
부산,10354475
서울,29298340
인천,5522486


In [15]:
지역_연도_인구 = 인구조사.filter(['지역','연도','인구'])
지역_연도_인구

,지역,연도,인구
0,수도권,2015,9904312
1,수도권,2010,9631482
2,수도권,2005,9762546
3,경상권,2015,3448737
4,경상권,2010,3393191
5,경상권,2005,3512547
6,수도권,2015,2890451
7,수도권,2010,2632035


#### 두 이상 컬럼을 그룹으로 묶은 후 집계 수행

In [16]:
지역_연도별_최대인구 = 지역_연도_인구.groupby(['지역','연도']).max()
지역_연도별_최대인구

인구
지역  연도           
경상권 2005  3512547
    2010  3393191
    2015  3448737
수도권 2005  9762546
    2010  9631482
    2015  9904312

In [17]:
지역_연도별_최대인구 = 지역_연도_인구.groupby(['연도','지역']).max()
지역_연도별_최대인구

인구
연도   지역          
2005 경상권  3512547
     수도권  9762546
2010 경상권  3393191
     수도권  9631482
2015 경상권  3448737
     수도권  9904312

#### 하나의 컬럼에 대해 여러 개의 집계함수 동시 사용

> `agg()` 함수에 집계함수의 이름을 문자열 원소를  갖는 리스트로 설정

In [18]:
도시인구 = 인구조사.filter(['도시','인구'])
도시인구.groupby('도시').agg(['min', 'max', 'sum','mean'])

인구                                 
        min      max       sum          mean
도시                                          
부산  3393191  3512547  10354475  3.451492e+06
서울  9631482  9904312  29298340  9.766113e+06
인천  2632035  2890451   5522486  2.761243e+06

#### 사용자 정의 함수 만들어서 적용하기

In [21]:
# 시리즈 객체를 파라미터로 받자
def my_range(x):
    return np.max(x) - np.min(x)

# 사용자 정의 함수의 이름은 문자열로 지정하지 않는다.
도시인구.groupby('도시').agg(['max', 'min', 'sum', 'mean', my_range])

인구                                          
        max      min       sum          mean my_range
도시                                                   
부산  3512547  3393191  10354475  3.451492e+06   119356
서울  9904312  9631482  29298340  9.766113e+06   272830
인천  2890451  2632035   5522486  2.761243e+06   258416